In [7]:
import pyautogui 
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variáveis globais para controlar a execução do código e páginas atuais
rodando = True
current_face_page = 1  # Página atual da face
current_mouth_page = 1  # Página atual da boca

# Coordenadas das setas para mudança de página
LEFT_ARROW_COORD = (1421, 648)
RIGHT_ARROW_COORD = (1566, 652)

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    pyautogui.getWindowsWithTitle(jogo)[0].moveTo(x, y)

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Atualização das estruturas de dados com páginas
faces = {
    "feliz": [((1501, 330), 1), ((1576, 548), 1), ((1420, 277), 1), ((1511, 376), 1), ((1588, 379), 1)],
    "triste": [((1502, 539), 1), ((1415, 596), 2), ((1498, 540), 2)],
    "neutro": [((1428, 433), 1), ((1418, 478), 1), ((1581, 437), 1), ((1493, 275), 1), ((1506, 273), 2), ((1566, 280), 2)],
    "surpreso": [((1427, 276), 2), ((1422, 331), 2), ((1424, 588), 1), ((1423, 545), 1), ((1580, 489), 1)]
}

mouths = {
    "feliz": [((1422, 278), 1), ((1577, 283), 1), ((1419, 330), 1), ((1579, 433), 1), ((1584, 485), 2), ((1579, 537), 2), ((1496, 594), 2)],
    "triste": [((1502, 279), 1), ((1500, 328), 1), ((1416, 492), 1), ((1500, 441), 1), ((1569, 641), 2)],
    "neutro": [((1504, 543), 2), ((1588, 330), 2)],
    "boca_aberta_feliz": [((1584, 279), 3), ((1498, 333), 3), ((1414, 388), 3), ((1589, 374), 3), ((1422, 486), 3), ((1588, 486), 3), ((1428, 573), 3), ((1490, 382), 4), ((1434, 594), 4)],
    "boca_aberta_surpreso": [((1586, 331), 3), ((1405, 325), 3), ((1583, 424), 3)],
    "surpreso": [((1502, 330), 1), ((1420, 400), 1), ((1500, 460), 1)],
    "boca_aberta_neutro": [((1577, 435), 3), ((1576, 330), 3)],
    "boca_aberta_triste": [((1577, 435), 3), ((1576, 330), 3)]
}

# Função para mudar de página na face
def switch_face_page(target_page):
    global current_face_page
    if current_face_page != target_page:
        difference = target_page - current_face_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.2)  # Pequeno delay para a página carregar
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.2)
        current_face_page = target_page
        time.sleep(0.5)  # Espera meio segundo para garantir que a página carregou

# Função para mudar de página na boca
def switch_mouth_page(target_page):
    global current_mouth_page
    if current_mouth_page != target_page:
        difference = target_page - current_mouth_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.2)
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.2)
        current_mouth_page = target_page
        time.sleep(0.5)  # Espera meio segundo para garantir que a página carregou

# Função para selecionar expressão facial e boca
def selecionar_face_mouth(emocao, boca_aberta=False):
    global current_face_page, current_mouth_page

    # Clicar no chat para iniciar a seleção da face e boca
    pyautogui.click(1591, 351)  # Clicar em face

    # Após clicar em face, a página reseta para 1
    current_face_page = 1

    # Selecionar face
    face_options = faces[emocao]
    face_coord, face_page = random.choice(face_options)
    switch_face_page(face_page)
    pyautogui.click(face_coord)  # Clicar na face

    # Clicar no botão de seleção da boca
    pyautogui.click(1492, 653)

    # Após clicar em mouth, a página reseta para 1
    current_mouth_page = 1

    # Selecionar boca
    if boca_aberta:
        mouth_key = f'boca_aberta_{emocao}'
        mouth_options = mouths.get(mouth_key, mouths.get(emocao, []))
    else:
        mouth_options = mouths.get(emocao, [])
    if not mouth_options:
        print(f"Nenhuma opção de boca encontrada para a emoção '{emocao}' com boca_aberta={boca_aberta}.")
        return
    mouth_coord, mouth_page = random.choice(mouth_options)
    switch_mouth_page(mouth_page)
    pyautogui.click(mouth_coord)  # Clicar na boca

# Função para processar cada personagem
def processar_personagem(personagem, coordenada_personagem, emocao, boca_aberta=False):
    print(f"Processando personagem: {personagem}")

    # Verificar se o personagem é Itadori para clicar duas vezes
    if personagem == "Itadori":
        print("Clicando duas vezes no Itadori...")
        pyautogui.click(coordenada_personagem)
        pyautogui.click(coordenada_personagem)
    else:
        pyautogui.click(coordenada_personagem)

    # Selecionar face e boca
    selecionar_face_mouth(emocao, boca_aberta=boca_aberta)

    # Clicar duas vezes na área para confirmar
    pyautogui.click(1573, 217)
    pyautogui.click(1573, 217)

# Função para clicar no botão "view"
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"

# Função para processar todas as falas
def processar_falas(falas_emocoes):
    personagens = [
        {"nome": "Itadori", "coordenada": (1659, 210)},
        {"nome": "Nobara", "coordenada": (1659, 259)},
        {"nome": "Megumi", "coordenada": (1661, 306)},
        {"nome": "Maki", "coordenada": (1660, 355)},
        {"nome": "Toge", "coordenada": (1657, 403)},
        {"nome": "Gojo", "coordenada": (1657, 450)}
    ]

    total_prints = 0
    max_prints = 300

    while total_prints < max_prints and rodando:
        for fala_emocao in falas_emocoes:
            if not rodando:
                break

            # Escolher qual personagem terá a boca aberta
            personagem_boca_aberta = random.choice(personagens)
            emocao = fala_emocao[1]

            # Processar cada personagem
            for personagem in personagens:
                boca_aberta = personagem == personagem_boca_aberta
                processar_personagem(personagem["nome"], personagem["coordenada"], emocao, boca_aberta)

            # Clicar no botão "view"
            clicar_view()

            # Capturar o print da área "view"
            capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
            total_prints += 1

            if total_prints >= max_prints:
                break

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            if "(" in linha:
                fala, emocao = linha.rsplit("(", 1)
                emocao = emocao.replace(")", "").replace("\"", "").strip()
                falas_emocoes.append((fala.strip(), emocao))
            else:
                falas_emocoes.append((linha.strip(), "neutro"))  # Emoção padrão se não especificada
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Nebula", 800, 150)

        pyautogui.click(1390, 1060)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando personagem: Itadori
Clicando duas vezes no Itadori...
Processando personagem: Nobara
Processando personagem: Megumi
Processando personagem: Maki
Processando personagem: Toge
Processando personagem: Gojo
Clicando no botão 'view'...
Processando personagem: Itadori
Clicando duas vezes no Itadori...
Processando personagem: Nobara
Processando personagem: Megumi
Processando personagem: Maki
Processando personagem: Toge
Processando personagem: Gojo
Clicando no botão 'view'...
Processando personagem: Itadori
Clicando duas vezes no Itadori...
Processando personagem: Nobara
Processando personagem: Megumi
Processando personagem: Maki
Processando personagem: Toge
Processando personagem: Gojo
Clicando no botão 'view'...
Processando personagem: Itadori
Clicando duas vezes no Itadori...
Execução interrompida pelo usuário (ESC).
Processando personagem: Nobara
Processando personagem: Megumi
Processando personagem: Maki
Processando personagem: Toge
Processando personagem: Gojo
Clicando no bo

Sincronização das emoções dos personagens

In [8]:
import pyautogui 
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variáveis globais para controlar a execução do código e páginas atuais
rodando = True
current_face_page = 1  # Página atual da face
current_mouth_page = 1  # Página atual da boca

# Coordenadas das setas para mudança de página
LEFT_ARROW_COORD = (1421, 648)
RIGHT_ARROW_COORD = (1566, 652)

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    pyautogui.getWindowsWithTitle(jogo)[0].moveTo(x, y)

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Atualização das estruturas de dados com páginas
faces = {
    "feliz": [((1501, 330), 1), ((1576, 548), 1), ((1420, 277), 1), ((1511, 376), 1), ((1588, 379), 1)],
    "triste": [((1502, 539), 1), ((1415, 596), 2), ((1498, 540), 2)],
    "neutro": [((1428, 433), 1), ((1418, 478), 1), ((1581, 437), 1), ((1493, 275), 1), ((1506, 273), 2), ((1566, 280), 2)],
    "surpreso": [((1427, 276), 2), ((1422, 331), 2), ((1424, 588), 1), ((1423, 545), 1), ((1580, 489), 1)]
}

mouths = {
    "feliz": [((1422, 278), 1), ((1577, 283), 1), ((1419, 330), 1), ((1579, 433), 1), ((1584, 485), 2), ((1579, 537), 2), ((1496, 594), 2)],
    "triste": [((1502, 279), 1), ((1500, 328), 1), ((1416, 492), 1), ((1500, 441), 1), ((1569, 641), 2)],
    "neutro": [((1504, 543), 2), ((1588, 330), 2)],
    "boca_aberta_feliz": [((1584, 279), 3), ((1498, 333), 3), ((1414, 388), 3), ((1589, 374), 3), ((1422, 486), 3), ((1588, 486), 3), ((1428, 573), 3), ((1490, 382), 4), ((1434, 594), 4)],
    "boca_aberta_surpreso": [((1586, 331), 3), ((1405, 325), 3), ((1583, 424), 3)],
    "surpreso": [((1502, 330), 1), ((1420, 400), 1), ((1500, 460), 1)],
    "boca_aberta_neutro": [((1577, 435), 3), ((1576, 330), 3)],
    "boca_aberta_triste": [((1577, 435), 3), ((1576, 330), 3)]
}

# Função para mudar de página na face
def switch_face_page(target_page):
    global current_face_page
    if current_face_page != target_page:
        difference = target_page - current_face_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.2)
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.2)
        current_face_page = target_page
        time.sleep(0.5)

# Função para mudar de página na boca
def switch_mouth_page(target_page):
    global current_mouth_page
    if current_mouth_page != target_page:
        difference = target_page - current_mouth_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.2)
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.2)
        current_mouth_page = target_page
        time.sleep(0.5)

# Função para selecionar expressão facial e boca
def selecionar_face_mouth(emocao, boca_aberta=False):
    global current_face_page, current_mouth_page

    # Clicar no chat para iniciar a seleção da face e boca
    pyautogui.click(1591, 351)  # Clicar em face

    # Após clicar em face, a página reseta para 1
    current_face_page = 1

    # Selecionar face
    face_options = faces.get(emocao, faces["neutro"])
    face_coord, face_page = random.choice(face_options)
    switch_face_page(face_page)
    pyautogui.click(face_coord)  # Clicar na face

    # Clicar no botão de seleção da boca
    pyautogui.click(1492, 653)

    # Após clicar em mouth, a página reseta para 1
    current_mouth_page = 1

    # Selecionar boca
    if boca_aberta:
        mouth_key = f'boca_aberta_{emocao}'
        mouth_options = mouths.get(mouth_key, mouths.get(emocao, []))
    else:
        mouth_options = mouths.get(emocao, [])
    if not mouth_options:
        print(f"Nenhuma opção de boca encontrada para a emoção '{emocao}' com boca_aberta={boca_aberta}.")
        return
    mouth_coord, mouth_page = random.choice(mouth_options)
    switch_mouth_page(mouth_page)
    pyautogui.click(mouth_coord)  # Clicar na boca

# Função para processar cada personagem
def processar_personagem(personagem, coordenada_personagem, emocao, boca_aberta=False):
    print(f"Processando personagem: {personagem} - Boca aberta: {boca_aberta}")

    # Verificar se o personagem é Itadori para clicar duas vezes
    if personagem == "Itadori":
        print("Clicando duas vezes no Itadori...")
        pyautogui.click(coordenada_personagem)
        pyautogui.click(coordenada_personagem)
    else:
        pyautogui.click(coordenada_personagem)

    # Selecionar face e boca
    selecionar_face_mouth(emocao, boca_aberta=boca_aberta)

    # Clicar duas vezes na área para confirmar
    pyautogui.click(1573, 217)
    pyautogui.click(1573, 217)

# Função para clicar no botão "view"
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"

# Função para processar todas as falas
def processar_falas(falas_emocoes):
    personagens = [
        {"nome": "Itadori", "coordenada": (1659, 210)},
        {"nome": "Nobara", "coordenada": (1659, 259)},
        {"nome": "Megumi", "coordenada": (1661, 306)},
        {"nome": "Maki", "coordenada": (1660, 355)},
        {"nome": "Toge", "coordenada": (1657, 403)},
        {"nome": "Gojo", "coordenada": (1657, 450)}
    ]

    total_prints = 0
    max_prints = len(falas_emocoes)

    for fala_emocao in falas_emocoes:
        if not rodando:
            break

        personagem_fala = fala_emocao["personagem"]
        emocao = fala_emocao["emocao"]

        print(f"Processando fala: '{fala_emocao['fala']}' ({emocao}) - Personagem: {personagem_fala}")

        # Processar cada personagem
        for personagem in personagens:
            # Definir boca aberta apenas para o personagem da fala
            boca_aberta = personagem["nome"] == personagem_fala
            processar_personagem(personagem["nome"], personagem["coordenada"], emocao, boca_aberta)

        # Clicar no botão "view"
        clicar_view()

        # Capturar o print da área "view"
        capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
        total_prints += 1

        if total_prints >= max_prints:
            break

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue  # Pular linhas vazias
            if ":" in linha and "(" in linha:
                personagem_part, resto = linha.split(":", 1)
                personagem = personagem_part.strip()
                fala_part, emocao_part = resto.rsplit("(", 1)
                fala = fala_part.strip()
                emocao = emocao_part.replace(")", "").strip()
                falas_emocoes.append({
                    "personagem": personagem,
                    "fala": fala,
                    "emocao": emocao
                })
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Nebula", 800, 150)

        pyautogui.click(1390, 1060)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando fala: 'Quem?' (feliz) - Personagem: Nobara
Processando personagem: Itadori - Boca aberta: False
Clicando duas vezes no Itadori...
Processando personagem: Nobara - Boca aberta: True
Processando personagem: Megumi - Boca aberta: False
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...
Processando fala: 'Interessante.' (neutro) - Personagem: Megumi
Processando personagem: Itadori - Boca aberta: False
Clicando duas vezes no Itadori...
Processando personagem: Nobara - Boca aberta: False
Processando personagem: Megumi - Boca aberta: True
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...
Processando fala: 'Vamos lá!' (feliz) - Personagem: Itadori
Processando personagem: Itadori - Boca aberta: True
Clicando duas vezes no Itadori...

O Problema:
Estado da Página Salvo por Personagem: Quando processamos um personagem, as páginas de face e boca permanecem nas últimas usadas para aquele personagem.
Nosso Código Atual: Supõe que, após clicar em face ou boca, a página reseta para 1, o que não é o caso.
Solução Proposta:
Para resolver esse problema, precisamos garantir que, ao processar cada personagem:

Resetar as Páginas de Face e Boca para 1:

Antes de selecionar a face e a boca, resetamos as páginas para 1.
Fazemos isso clicando no botão de seta esquerda o número máximo de vezes necessário para chegar à página 1.
Atualizar as Funções selecionar_face_mouth, reset_face_page e reset_mouth_page:

As funções agora resetam as páginas antes de selecionar as expressões.
Isso garante que o script sempre saiba em qual página está, independentemente do estado salvo do personagem.

In [9]:
import pyautogui 
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variáveis globais para controlar a execução do código e páginas atuais
rodando = True
current_face_page = 1  # Página atual da face
current_mouth_page = 1  # Página atual da boca

# Coordenadas das setas para mudança de página
LEFT_ARROW_COORD = (1421, 648)
RIGHT_ARROW_COORD = (1566, 652)

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    pyautogui.getWindowsWithTitle(jogo)[0].moveTo(x, y)

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Atualização das estruturas de dados com páginas
faces = {
    "feliz": [((1501, 330), 1), ((1576, 548), 1), ((1420, 277), 1), ((1511, 376), 1), ((1588, 379), 1)],
    "triste": [((1502, 539), 1), ((1415, 596), 2), ((1498, 540), 2)],
    "neutro": [((1428, 433), 1), ((1418, 478), 1), ((1581, 437), 1), ((1493, 275), 1), ((1506, 273), 2), ((1566, 280), 2)],
    "surpreso": [((1427, 276), 2), ((1422, 331), 2), ((1424, 588), 1), ((1423, 545), 1), ((1580, 489), 1)]
}

mouths = {
    "feliz": [((1422, 278), 1), ((1577, 283), 1), ((1419, 330), 1), ((1579, 433), 1), ((1584, 485), 2), ((1579, 537), 2), ((1496, 594), 2)],
    "triste": [((1502, 279), 1), ((1500, 328), 1), ((1416, 492), 1), ((1500, 441), 1), ((1569, 641), 2)],
    "neutro": [((1504, 543), 2), ((1588, 330), 2)],
    "boca_aberta_feliz": [((1584, 279), 3), ((1498, 333), 3), ((1414, 388), 3), ((1589, 374), 3), ((1422, 486), 3), ((1588, 486), 3), ((1428, 573), 3), ((1490, 382), 4), ((1434, 594), 4)],
    "boca_aberta_surpreso": [((1586, 331), 3), ((1405, 325), 3), ((1583, 424), 3)],
    "surpreso": [((1502, 330), 1), ((1420, 400), 1), ((1500, 460), 1)],
    "boca_aberta_neutro": [((1577, 435), 3), ((1576, 330), 3)],
    "boca_aberta_triste": [((1577, 435), 3), ((1576, 330), 3)]
}

# Função para resetar a página de face para 1
def reset_face_page():
    global current_face_page
    max_face_pages = 2  # Número total de páginas de face
    # Clicar na seta esquerda (max_face_pages - 1) vezes para garantir que estamos na página 1
    for _ in range(max_face_pages - 1):
        pyautogui.click(LEFT_ARROW_COORD)
        time.sleep(0.2)
    current_face_page = 1

# Função para resetar a página de boca para 1
def reset_mouth_page():
    global current_mouth_page
    max_mouth_pages = 15  # Número total de páginas de boca
    # Clicar na seta esquerda (max_mouth_pages - 1) vezes para garantir que estamos na página 1
    for _ in range(max_mouth_pages - 1):
        pyautogui.click(LEFT_ARROW_COORD)
        time.sleep(0.1)
    current_mouth_page = 1

# Função para mudar de página na face
def switch_face_page(target_page):
    global current_face_page
    if current_face_page != target_page:
        difference = target_page - current_face_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.2)
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.2)
        current_face_page = target_page
        time.sleep(0.5)

# Função para mudar de página na boca
def switch_mouth_page(target_page):
    global current_mouth_page
    if current_mouth_page != target_page:
        difference = target_page - current_mouth_page
        if difference > 0:
            for _ in range(difference):
                pyautogui.click(RIGHT_ARROW_COORD)
                time.sleep(0.1)
        elif difference < 0:
            for _ in range(abs(difference)):
                pyautogui.click(LEFT_ARROW_COORD)
                time.sleep(0.1)
        current_mouth_page = target_page
        time.sleep(0.5)

# Função para selecionar expressão facial e boca
def selecionar_face_mouth(emocao, boca_aberta=False):
    global current_face_page, current_mouth_page

    # Clicar no chat para iniciar a seleção da face
    pyautogui.click(1591, 351)  # Clicar em face

    # Resetar a página de face para 1
    reset_face_page()

    # Selecionar face
    face_options = faces.get(emocao, faces["neutro"])
    face_coord, face_page = random.choice(face_options)
    switch_face_page(face_page)
    pyautogui.click(face_coord)  # Clicar na face

    # Clicar no botão de seleção da boca
    pyautogui.click(1492, 653)

    # Resetar a página de boca para 1
    reset_mouth_page()

    # Selecionar boca
    if boca_aberta:
        mouth_key = f'boca_aberta_{emocao}'
        mouth_options = mouths.get(mouth_key, mouths.get(emocao, []))
    else:
        mouth_options = mouths.get(emocao, [])
    if not mouth_options:
        print(f"Nenhuma opção de boca encontrada para a emoção '{emocao}' com boca_aberta={boca_aberta}.")
        return
    mouth_coord, mouth_page = random.choice(mouth_options)
    switch_mouth_page(mouth_page)
    pyautogui.click(mouth_coord)  # Clicar na boca

# Função para processar cada personagem
def processar_personagem(personagem, coordenada_personagem, emocao, boca_aberta=False):
    print(f"Processando personagem: {personagem} - Boca aberta: {boca_aberta}")

    # Clicar no personagem para selecioná-lo
    pyautogui.click(coordenada_personagem)

    # Selecionar face e boca
    selecionar_face_mouth(emocao, boca_aberta=boca_aberta)

    # Clicar duas vezes na área para confirmar
    pyautogui.click(1573, 217)
    pyautogui.click(1573, 217)

# Função para clicar no botão "view"
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"

# Função para processar todas as falas
def processar_falas(falas_emocoes):
    personagens = [
        {"nome": "Itadori", "coordenada": (1659, 210)},
        {"nome": "Nobara", "coordenada": (1659, 259)},
        {"nome": "Megumi", "coordenada": (1661, 306)},
        {"nome": "Maki", "coordenada": (1660, 355)},
        {"nome": "Toge", "coordenada": (1657, 403)},
        {"nome": "Gojo", "coordenada": (1657, 450)}
    ]

    total_prints = 0
    max_prints = len(falas_emocoes)

    for fala_emocao in falas_emocoes:
        if not rodando:
            break

        personagem_fala = fala_emocao["personagem"]
        emocao = fala_emocao["emocao"]

        print(f"Processando fala: '{fala_emocao['fala']}' ({emocao}) - Personagem: {personagem_fala}")

        # Processar cada personagem
        for personagem in personagens:
            # Definir boca aberta apenas para o personagem da fala
            boca_aberta = personagem["nome"] == personagem_fala
            processar_personagem(personagem["nome"], personagem["coordenada"], emocao, boca_aberta)

        # Clicar no botão "view"
        clicar_view()

        # Capturar o print da área "view"
        capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
        total_prints += 1

        if total_prints >= max_prints:
            break

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue  # Pular linhas vazias
            if ":" in linha and "(" in linha:
                personagem_part, resto = linha.split(":", 1)
                personagem = personagem_part.strip()
                fala_part, emocao_part = resto.rsplit("(", 1)
                fala = fala_part.strip()
                emocao = emocao_part.replace(")", "").strip()
                falas_emocoes.append({
                    "personagem": personagem,
                    "fala": fala,
                    "emocao": emocao
                })
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Nebula", 800, 150)

        pyautogui.click(1390, 1060)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando fala: 'Quem?' (feliz) - Personagem: Nobara
Processando personagem: Itadori - Boca aberta: False
Processando personagem: Nobara - Boca aberta: True
Processando personagem: Megumi - Boca aberta: False
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...
Processando fala: 'Interessante.' (neutro) - Personagem: Megumi
Processando personagem: Itadori - Boca aberta: False
Processando personagem: Nobara - Boca aberta: False
Processando personagem: Megumi - Boca aberta: True
Processando personagem: Maki - Boca aberta: False
Execução interrompida pelo usuário (ESC).
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...


In [11]:
import pyautogui 
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variáveis globais para controlar a execução do código
rodando = True

# Coordenadas das setas para mudança de página
LEFT_ARROW_COORD = (1421, 648)
RIGHT_ARROW_COORD = (1566, 652)

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    pyautogui.getWindowsWithTitle(jogo)[0].moveTo(x, y)

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Atualização das estruturas de dados com páginas
faces = {
    "feliz": [((1501, 330), 1), ((1576, 548), 1), ((1420, 277), 1), ((1511, 376), 1), ((1588, 379), 1)],
    "triste": [((1502, 539), 1), ((1415, 596), 2), ((1498, 540), 2)],
    "neutro": [((1428, 433), 1), ((1418, 478), 1), ((1581, 437), 1), ((1493, 275), 1), ((1506, 273), 2), ((1566, 280), 2)],
    "surpreso": [((1427, 276), 2), ((1422, 331), 2), ((1424, 588), 1), ((1423, 545), 1), ((1580, 489), 1)]
}

mouths = {
    "feliz": [((1422, 278), 1), ((1577, 283), 1), ((1419, 330), 1), ((1579, 433), 1), ((1584, 485), 2), ((1579, 537), 2), ((1496, 594), 2)],
    "triste": [((1502, 279), 1), ((1500, 328), 1), ((1416, 492), 1), ((1500, 441), 1), ((1569, 641), 2)],
    "neutro": [((1504, 543), 2), ((1588, 330), 2)],
    "boca_aberta_feliz": [((1584, 279), 3), ((1498, 333), 3), ((1414, 388), 3), ((1589, 374), 3), ((1422, 486), 3), ((1588, 486), 3), ((1428, 573), 3), ((1490, 382), 4), ((1434, 594), 4)],
    "boca_aberta_surpreso": [((1586, 331), 3), ((1405, 325), 3), ((1583, 424), 3)],
    "surpreso": [((1502, 330), 1), ((1420, 400), 1), ((1500, 460), 1)],
    "boca_aberta_neutro": [((1577, 435), 3), ((1576, 330), 3)],
    "boca_aberta_triste": [((1577, 435), 3), ((1576, 330), 3)]
}

# Lista de personagens com páginas atuais
personagens = [
    {"nome": "Itadori", "coordenada": (1659, 210), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Nobara", "coordenada": (1659, 259), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Megumi", "coordenada": (1661, 306), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Maki", "coordenada": (1660, 355), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Toge", "coordenada": (1657, 403), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Gojo", "coordenada": (1657, 450), "current_face_page": None, "current_mouth_page": None}
]

# Funções de reset e mudança de página
def reset_face_page(personagem):
    switch_face_page(personagem, 1)

def reset_mouth_page(personagem):
    switch_mouth_page(personagem, 1)

def switch_face_page(personagem, target_page):
    current_page = personagem['current_face_page'] or 1
    num_pages = 2

    if current_page == target_page:
        return

    forward_clicks = (target_page - current_page) % num_pages
    backward_clicks = (current_page - target_page) % num_pages

    if forward_clicks <= backward_clicks:
        for _ in range(forward_clicks):
            pyautogui.click(RIGHT_ARROW_COORD)
            time.sleep(0.2)
    else:
        for _ in range(backward_clicks):
            pyautogui.click(LEFT_ARROW_COORD)
            time.sleep(0.2)
    personagem['current_face_page'] = target_page
    time.sleep(0.5)

def switch_mouth_page(personagem, target_page):
    current_page = personagem['current_mouth_page'] or 1
    num_pages = 15

    if current_page == target_page:
        return

    forward_clicks = (target_page - current_page) % num_pages
    backward_clicks = (current_page - target_page) % num_pages

    if forward_clicks <= backward_clicks:
        for _ in range(forward_clicks):
            pyautogui.click(RIGHT_ARROW_COORD)
            time.sleep(0.1)
    else:
        for _ in range(backward_clicks):
            pyautogui.click(LEFT_ARROW_COORD)
            time.sleep(0.1)
    personagem['current_mouth_page'] = target_page
    time.sleep(0.5)

# Função para selecionar expressão facial e boca
def selecionar_face_mouth(personagem, emocao, boca_aberta=False):
    # Clicar no chat para iniciar a seleção da face
    pyautogui.click(1591, 351)  # Clicar em face

    # Resetar a página de face para 1
    reset_face_page(personagem)

    # Selecionar face
    face_options = faces.get(emocao, faces["neutro"])
    face_coord, face_page = random.choice(face_options)
    switch_face_page(personagem, face_page)
    pyautogui.click(face_coord)  # Clicar na face

    # Clicar no botão de seleção da boca
    pyautogui.click(1492, 653)

    # Resetar a página de boca para 1
    reset_mouth_page(personagem)

    # Selecionar boca
    if boca_aberta:
        mouth_key = f'boca_aberta_{emocao}'
        mouth_options = mouths.get(mouth_key, mouths.get(emocao, []))
    else:
        mouth_options = mouths.get(emocao, [])
    if not mouth_options:
        print(f"Nenhuma opção de boca encontrada para a emoção '{emocao}' com boca_aberta={boca_aberta}.")
        return
    mouth_coord, mouth_page = random.choice(mouth_options)
    switch_mouth_page(personagem, mouth_page)
    pyautogui.click(mouth_coord)  # Clicar na boca

# Função para processar cada personagem
def processar_personagem(personagem, emocao, boca_aberta=False):
    print(f"Processando personagem: {personagem['nome']} - Boca aberta: {boca_aberta}")

    # Clicar no personagem para selecioná-lo
    pyautogui.click(personagem['coordenada'])

    # Selecionar face e boca
    selecionar_face_mouth(personagem, emocao, boca_aberta=boca_aberta)

    # Clicar duas vezes na área para confirmar
    pyautogui.click(1573, 217)
    pyautogui.click(1573, 217)

# Função para clicar no botão "view"
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"

# Função para processar todas as falas
def processar_falas(falas_emocoes):
    total_prints = 0
    max_prints = len(falas_emocoes)

    for fala_emocao in falas_emocoes:
        if not rodando:
            break

        personagem_fala = fala_emocao["personagem"]
        emocao = fala_emocao["emocao"]

        print(f"Processando fala: '{fala_emocao['fala']}' ({emocao}) - Personagem: {personagem_fala}")

        # Processar cada personagem
        for personagem in personagens:
            # Definir boca aberta apenas para o personagem da fala
            boca_aberta = personagem["nome"] == personagem_fala
            processar_personagem(personagem, emocao, boca_aberta)

        # Clicar no botão "view"
        clicar_view()

        # Capturar o print da área "view"
        capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
        total_prints += 1

        if total_prints >= max_prints:
            break

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue  # Pular linhas vazias
            if ":" in linha and "(" in linha:
                personagem_part, resto = linha.split(":", 1)
                personagem = personagem_part.strip()
                fala_part, emocao_part = resto.rsplit("(", 1)
                fala = fala_part.strip()
                emocao = emocao_part.replace(")", "").strip()
                falas_emocoes.append({
                    "personagem": personagem,
                    "fala": fala,
                    "emocao": emocao
                })
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Nebula", 800, 150)

        pyautogui.click(1390, 1060)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando fala: 'Quem?' (feliz) - Personagem: Nobara
Processando personagem: Itadori - Boca aberta: False
Processando personagem: Nobara - Boca aberta: True
Execução interrompida pelo usuário (ESC).
Processando personagem: Megumi - Boca aberta: False
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...


Ajuste itadori clicar 2x

In [12]:
import pyautogui 
import time
import random
import os
from PIL import ImageGrab
from pynput.keyboard import Listener as KeyboardListener, Key

# Variáveis globais para controlar a execução do código
rodando = True

# Coordenadas das setas para mudança de página
LEFT_ARROW_COORD = (1421, 648)
RIGHT_ARROW_COORD = (1566, 652)

# Função para parar a execução ao pressionar ESC
def on_press(key):
    global rodando
    if key == Key.esc:
        print("Execução interrompida pelo usuário (ESC).")
        rodando = False
        return False  # Para o listener de teclado

# Função para mover a janela do jogo
def mover_janela(jogo, x, y):
    windows = pyautogui.getWindowsWithTitle(jogo)
    if windows:
        windows[0].moveTo(x, y)
    else:
        print(f"Janela com título '{jogo}' não encontrada.")

# Função para capturar prints
def capturar_print(nome_arquivo, coordenadas):
    if not os.path.exists("./prints"):
        os.makedirs("./prints")
    imagem = ImageGrab.grab(bbox=coordenadas)
    imagem.save(f"./prints/{nome_arquivo}.png")

# Atualização das estruturas de dados com páginas
faces = {
    "feliz": [((1501, 330), 1), ((1576, 548), 1), ((1420, 277), 1), ((1511, 376), 1), ((1588, 379), 1)],
    "triste": [((1502, 539), 1), ((1415, 596), 2), ((1498, 540), 2)],
    "neutro": [((1428, 433), 1), ((1418, 478), 1), ((1581, 437), 1), ((1493, 275), 1), ((1506, 273), 2), ((1566, 280), 2)],
    "surpreso": [((1427, 276), 2), ((1422, 331), 2), ((1424, 588), 1), ((1423, 545), 1), ((1580, 489), 1)]
}

mouths = {
    "feliz": [((1422, 278), 1), ((1577, 283), 1), ((1419, 330), 1), ((1579, 433), 1), ((1584, 485), 2), ((1579, 537), 2), ((1496, 594), 2)],
    "triste": [((1502, 279), 1), ((1500, 328), 1), ((1416, 492), 1), ((1500, 441), 1), ((1569, 641), 2)],
    "neutro": [((1504, 543), 2), ((1588, 330), 2)],
    "boca_aberta_feliz": [((1584, 279), 3), ((1498, 333), 3), ((1414, 388), 3), ((1589, 374), 3), ((1422, 486), 3), ((1588, 486), 3), ((1428, 573), 3), ((1490, 382), 4), ((1434, 594), 4)],
    "boca_aberta_surpreso": [((1586, 331), 3), ((1405, 325), 3), ((1583, 424), 3)],
    "surpreso": [((1502, 330), 1), ((1420, 400), 1), ((1500, 460), 1)],
    "boca_aberta_neutro": [((1577, 435), 3), ((1576, 330), 3)],
    "boca_aberta_triste": [((1577, 435), 3), ((1576, 330), 3)]
}

# Lista de personagens com páginas atuais
personagens = [
    {"nome": "Itadori", "coordenada": (1659, 210), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Nobara", "coordenada": (1659, 259), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Megumi", "coordenada": (1661, 306), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Maki", "coordenada": (1660, 355), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Toge", "coordenada": (1657, 403), "current_face_page": None, "current_mouth_page": None},
    {"nome": "Gojo", "coordenada": (1657, 450), "current_face_page": None, "current_mouth_page": None}
]

# Funções de reset e mudança de página
def reset_face_page(personagem):
    switch_face_page(personagem, 1)

def reset_mouth_page(personagem):
    switch_mouth_page(personagem, 1)

def switch_face_page(personagem, target_page):
    current_page = personagem['current_face_page'] or 1
    num_pages = 2

    if current_page == target_page:
        return

    forward_clicks = (target_page - current_page) % num_pages
    backward_clicks = (current_page - target_page) % num_pages

    if forward_clicks <= backward_clicks:
        for _ in range(forward_clicks):
            pyautogui.click(RIGHT_ARROW_COORD)
            time.sleep(0.2)
    else:
        for _ in range(backward_clicks):
            pyautogui.click(LEFT_ARROW_COORD)
            time.sleep(0.2)
    personagem['current_face_page'] = target_page
    time.sleep(0.5)

def switch_mouth_page(personagem, target_page):
    current_page = personagem['current_mouth_page'] or 1
    num_pages = 15

    if current_page == target_page:
        return

    forward_clicks = (target_page - current_page) % num_pages
    backward_clicks = (current_page - target_page) % num_pages

    if forward_clicks <= backward_clicks:
        for _ in range(forward_clicks):
            pyautogui.click(RIGHT_ARROW_COORD)
            time.sleep(0.1)
    else:
        for _ in range(backward_clicks):
            pyautogui.click(LEFT_ARROW_COORD)
            time.sleep(0.1)
    personagem['current_mouth_page'] = target_page
    time.sleep(0.5)

# Função para selecionar expressão facial e boca
def selecionar_face_mouth(personagem, emocao, boca_aberta=False):
    # Clicar no chat para iniciar a seleção da face
    pyautogui.click(1591, 351)  # Clicar em face

    # Resetar a página de face para 1
    reset_face_page(personagem)

    # Selecionar face
    face_options = faces.get(emocao, faces["neutro"])
    face_coord, face_page = random.choice(face_options)
    switch_face_page(personagem, face_page)
    pyautogui.click(face_coord)  # Clicar na face

    # Clicar no botão de seleção da boca
    pyautogui.click(1492, 653)

    # Resetar a página de boca para 1
    reset_mouth_page(personagem)

    # Selecionar boca
    if boca_aberta:
        mouth_key = f'boca_aberta_{emocao}'
        mouth_options = mouths.get(mouth_key, mouths.get(emocao, []))
    else:
        mouth_options = mouths.get(emocao, [])
    if not mouth_options:
        print(f"Nenhuma opção de boca encontrada para a emoção '{emocao}' com boca_aberta={boca_aberta}.")
        return
    mouth_coord, mouth_page = random.choice(mouth_options)
    switch_mouth_page(personagem, mouth_page)
    pyautogui.click(mouth_coord)  # Clicar na boca

# Função para processar cada personagem
def processar_personagem(personagem, emocao, boca_aberta=False):
    print(f"Processando personagem: {personagem['nome']} - Boca aberta: {boca_aberta}")

    # Verificar se o personagem é Itadori para clicar duas vezes
    if personagem['nome'] == "Itadori":
        print("Clicando duas vezes no Itadori...")
        pyautogui.click(personagem['coordenada'])
        pyautogui.click(personagem['coordenada'])
    else:
        pyautogui.click(personagem['coordenada'])

    # Selecionar face e boca
    selecionar_face_mouth(personagem, emocao, boca_aberta=boca_aberta)

    # Clicar duas vezes na área para confirmar
    pyautogui.click(1573, 217)
    pyautogui.click(1573, 217)

# Função para clicar no botão "view"
def clicar_view():
    print("Clicando no botão 'view'...")
    pyautogui.click(859, 631)  # Coordenada do botão "view"

# Função para processar todas as falas
def processar_falas(falas_emocoes):
    total_prints = 0
    max_prints = len(falas_emocoes)

    for fala_emocao in falas_emocoes:
        if not rodando:
            break

        personagem_fala = fala_emocao["personagem"]
        emocao = fala_emocao["emocao"]

        print(f"Processando fala: '{fala_emocao['fala']}' ({emocao}) - Personagem: {personagem_fala}")

        # Processar cada personagem
        for personagem in personagens:
            # Definir boca aberta apenas para o personagem da fala
            boca_aberta = personagem["nome"] == personagem_fala
            processar_personagem(personagem, emocao, boca_aberta)

        # Clicar no botão "view"
        clicar_view()

        # Capturar o print da área "view"
        capturar_print(f"print_view_{total_prints + 1}", (809, 184, 1703, 675))
        total_prints += 1

        if total_prints >= max_prints:
            break

# Ler falas do arquivo txt
def ler_falas(arquivo):
    falas_emocoes = []
    with open(arquivo, "r", encoding="utf-8") as f:
        for linha in f:
            linha = linha.strip()
            if not linha:
                continue  # Pular linhas vazias
            if ":" in linha and "(" in linha:
                personagem_part, resto = linha.split(":", 1)
                personagem = personagem_part.strip()
                fala_part, emocao_part = resto.rsplit("(", 1)
                fala = fala_part.strip()
                emocao = emocao_part.replace(")", "").strip()
                falas_emocoes.append({
                    "personagem": personagem,
                    "fala": fala,
                    "emocao": emocao
                })
    return falas_emocoes

# Função principal
def main():
    # Iniciar o listener do teclado para ESC
    with KeyboardListener(on_press=on_press) as listener:
        mover_janela("Gacha Nebula", 800, 150)

        # Esperar um pouco para garantir que a janela foi movida
        time.sleep(1)

        # Clicar na posição inicial, se necessário
        pyautogui.click(1390, 1060)

        falas_emocoes = ler_falas("falas.txt")
        processar_falas(falas_emocoes)

        listener.join()  # Espera o listener de teclado encerrar

if __name__ == "__main__":
    main()


Processando fala: 'Quem?' (feliz) - Personagem: Nobara
Processando personagem: Itadori - Boca aberta: False
Clicando duas vezes no Itadori...
Processando personagem: Nobara - Boca aberta: True
Processando personagem: Megumi - Boca aberta: False
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...
Processando fala: 'Interessante.' (neutro) - Personagem: Megumi
Processando personagem: Itadori - Boca aberta: False
Clicando duas vezes no Itadori...
Processando personagem: Nobara - Boca aberta: False
Processando personagem: Megumi - Boca aberta: True
Processando personagem: Maki - Boca aberta: False
Processando personagem: Toge - Boca aberta: False
Processando personagem: Gojo - Boca aberta: False
Clicando no botão 'view'...
Processando fala: 'Vamos lá!' (feliz) - Personagem: Itadori
Processando personagem: Itadori - Boca aberta: True
Clicando duas vezes no Itadori...